In [8]:
import torch
from torch import nn
from torchvision import models
from collections import OrderedDict
import numpy as np
import itertools

import warnings
warnings.filterwarnings('ignore') 

import warnings
warnings.filterwarnings('ignore') 
# Load your model to this variable
model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(1024, 512)), # model.classifier.in_features
                                              ('relu', nn.ReLU()),
                                              ('dropout1', nn.Dropout(0.2)), # To avoid over-fitting
                                              ('fc2', nn.Linear(512, 102)),
                                              ('output', nn.LogSoftmax(dim=1))
                                             ]))

# Upload checkpoint saved from local PyTorch version 0.4.1
checkpoint_path = 'model_e60.pth'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint, strict=False)
print("The state dict keys: \n\n", model.state_dict().keys())

The state dict keys: 



 odict_keys(['features.conv0.weight', 'features.norm0.weight', 'features.norm0.bias', 'features.norm0.running_mean', 'features.norm0.running_var', 'features.norm0.num_batches_tracked', 'features.denseblock1.denselayer1.norm1.weight', 'features.denseblock1.denselayer1.norm1.bias', 'features.denseblock1.denselayer1.norm1.running_mean', 'features.denseblock1.denselayer1.norm1.running_var', 'features.denseblock1.denselayer1.norm1.num_batches_tracked', 'features.denseblock1.denselayer1.conv1.weight', 'features.denseblock1.denselayer1.norm2.weight', 'features.denseblock1.denselayer1.norm2.bias', 'features.denseblock1.denselayer1.norm2.running_mean', 'features.denseblock1.denselayer1.norm2.running_var', 'features.denseblock1.denselayer1.norm2.num_batches_tracked', 'features.denseblock1.denselayer1.conv2.weight', 'features.denseblock1.denselayer2.norm1.weight', 'features.denseblock1.denselayer2.norm1.bias', 'features.denseblock1.denselayer2.norm1.running_mean', 'features.denseblock1.denselayer

In [9]:
# Load your model to this variable
model_colab = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model_colab.classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(1024, 512)), # model.classifier.in_features
                                                   ('relu', nn.ReLU()),
                                                   ('dropout1', nn.Dropout(0.2)), # To avoid over-fitting
                                                   ('fc2', nn.Linear(512, 102)),
                                                   ('output', nn.LogSoftmax(dim=1))
                                                  ]))

# Upload another checkpoint saved from Colab PyTorch version 0.4.0
checkpoint_path = 'model_colab.pth'
checkpoint = torch.load(checkpoint_path)
model_colab.load_state_dict(checkpoint, strict=False)
#print("The state dict keys: \n\n", model_colab.state_dict().keys())

Compare texts online: https://text-compare.com/ 
The two texts of model.state_dict().keys() are identical!

In [10]:
print(type(model.state_dict().keys()))
print(model.state_dict().keys() == model_colab.state_dict().keys())

<class 'odict_keys'>
True


In [11]:
print(type(model.state_dict().values()))
print(model.state_dict().values() == model_colab.state_dict().values())

<class 'odict_values'>
False


In [12]:
#print(model.state_dict().values())

In [13]:
print('Structure of state_dict:')
for idx, item in enumerate(list(model.state_dict().items())):
    print('    model      : {}, key: {}, value: {}'.format(idx, item[0], item[1].shape))
    item_colab = list(model_colab.state_dict().items())[idx]
    print('    model_colab: {}, key: {}, value: {}'.format(idx, item_colab[0], item_colab[1].shape))
    identical = torch.sum(item[1] == item_colab[1]).item()
    print('\n', identical, ' vs. 64*3*7*7 =', np.prod(item[1].shape))
    break

Structure of state_dict:
    model      : 0, key: features.conv0.weight, value: torch.Size([64, 3, 7, 7])
    model_colab: 0, key: features.conv0.weight, value: torch.Size([64, 3, 7, 7])

 9408  vs. 64*3*7*7 = 9408


In [16]:
diff = 0
for idx, item in enumerate(list(model.state_dict().items())):
    item_colab = list(model_colab.state_dict().items())[idx]
    identical = torch.sum(item[1] == item_colab[1]).item()
    if identical != np.prod(item[1].shape):
        diff += 1
print("Total number of nodes: ", len(list(model.state_dict().items())))
print("Number of different nodes: ", diff)

Total number of nodes:  729
Number of different nodes:  367
